In [ ]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [ ]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [ ]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [ ]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(2,2)],
              'vect__stop_words': [stopwords, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'vect__norm': ['l1', 'l2', None],
              'vect__use_idf':[False, True],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.1, 1.0, 10.0, 100.0]}

pipe = Pipeline([
                   ('vect', tfidf),
                   ('clf', LogisticRegression(random_state=42))
                   ])

model = RandomizedSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, n_iter=50, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
print(model.best_estimator_)

In [ ]:
print(model.best_params_)

In [ ]:
print(model.best_score_)

In [ ]:
y_pred = model.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(conf_matrix)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'tfidf_2gram_lr_pipe.pkl'), 'wb'),
           protocol=4)